In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip uninstall cython --y
!pip uninstall plotnine --y
!pip install cython==0.29.35
!pip install molmod

Found existing installation: Cython 3.0.12
Uninstalling Cython-3.0.12:
  Successfully uninstalled Cython-3.0.12
Found existing installation: plotnine 0.14.5
Uninstalling plotnine-0.14.5:
  Successfully uninstalled plotnine-0.14.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for molmod: filename=molmod-1.4.8-cp311-cp311-linux_x86_64.whl size=1980833 sha256=cba701e3a52282f5aef9437cc9cdf6dfad1bf3846f1f78ea4f8124c3ddf35b46
  Stored in directory: /root/.cache/pip/wheels/98/92/84/23679616891eb95192bde87ba09ce91602d44bc2b1ac0d93ba
Successfully built molmod


In [ ]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from molmod.molecules import Molecule

In [ ]:
import RankerModel

Instructions for updating:
non-resource variables are not supported in the long term


# Download Data
* QM7X train data

In [ ]:
!cp '/content/gdrive/MyDrive/treino_full.zip' treino.zip

In [ ]:
!unzip treino.zip

In [ ]:
diretorio_treino = "/content/treino/"
arquivos_treino = os.listdir(diretorio_treino)
print("Total molecules available for training:", len(arquivos_treino))

Total molecules available for training: 6591


# Choose the Property

In [ ]:
#prop = "HOMO_energy" ## mean
prop = "PBE0_energy"
#prop = 'Atomization_energy'
#prop = "LUMO_energy" #mean
#prop = "Scalar_dipole_moment"
#prop = "Kinetic_energy"
#prop = "HOMO_LUMO_gap" #mean
#prop = "TS_dispersion_energy"
#prop = "Classical_coulomb_energy_el_el"
#prop = "Exchange_energy"

# Loss function

In [ ]:
#loss_f="BinaryCrossentropy"
loss_f="MSE"

# Auxliary Functions

In [ ]:
def FeatureMatrix(simbols, Max_dim):
  elementos_dict = {'C':0, 'Cl':1, 'H':2, 'N':3, 'O':4, 'S':5} #QM7-X
  matrix_features = np.zeros(shape=(Max_dim, len(elementos_dict)), dtype=float, order='C')
  dimensao = len(simbols)
  for i in range(dimensao):
    r = elementos_dict[simbols[i]]
    matrix_features[i,r] = 1

  return matrix_features


def Euclidian_Distance(vec_a, vec_b):
  # Input are atoms position
  # output euclidean distance
  aux = vec_a - vec_b
  aux = aux**2
  return sum(aux)**0.5


def Distance_Matrix(coordinates, max_dim):
  matrix_dist = np.zeros(shape=(max_dim, max_dim), dtype=float, order='C')
  n_atoms = coordinates.shape[0]
  for i in range(n_atoms):
    for j in range(i, n_atoms):
      vec_a = coordinates[i]
      vec_b = coordinates[j]
      dist = Euclidian_Distance(vec_a, vec_b)
      matrix_dist[i,j] = dist
      matrix_dist[j,i] = dist

  return matrix_dist

# Load Data

In [ ]:
# Molecules will be splitted 90% for training, 10% for validation
frac_treino = 0.9

In [ ]:
def ExtractIsomerosConfomeros(val):
  val=val.replace(".xyz","")
  aux = val.split("-")
  return aux[2], aux[3]

In [ ]:
lst_features_treino = list()
lst_distancias_treino = list()
lst_mol_sizes_treino = list()
lst_target_treino = list()

lst_features_valid = list()
lst_distancias_valid = list()
lst_mol_sizes_valid = list()
lst_target_valid = list()

In [ ]:
lst_moleculas_treino = list()
lst_isomero_treino = list()
lst_conformero_treino = list()

lst_moleculas_valid = list()
lst_isomero_valid = list()
lst_conformero_valid = list()

In [ ]:
inicio = time.time()
for idx_aux in tqdm(range(int(len(arquivos_treino)))):
  arq = arquivos_treino[idx_aux]
  conjunto = np.random.choice(['treino', 'valid'], size=1, replace=True, p=[frac_treino, 1-frac_treino])[0]

  df_aux = pd.read_csv(diretorio_treino + arq + "/propriedades.csv")

  df_aux['isomero'] = df_aux['molecula_config'].apply(lambda x: ExtractIsomerosConfomeros(x)[0]).values
  df_aux['conformero'] = df_aux['molecula_config'].apply(lambda x: ExtractIsomerosConfomeros(x)[1]).values
  df_aux_drop_dup = df_aux[['isomero', 'conformero']].drop_duplicates()
  lst_files = list()
  for i in range(df_aux_drop_dup.shape[0]):
    aux_isomero = df_aux_drop_dup.iloc[i]['isomero']
    aux_conformero = df_aux_drop_dup.iloc[i]['conformero']

    str_query = "isomero == '" + aux_isomero + "' and conformero == '" + aux_conformero + "'"
    df_aux_strat = df_aux.query(str_query)
    lst_files_aux0 = df_aux_strat['molecula_config'].tolist()

    lst_files_aux1 = lst_files_aux0.copy()

    lst_files_stratified = np.random.choice(lst_files_aux1, replace=False, size=30).tolist()
    for c in lst_files_stratified:
      lst_files.append(c+".xyz")


  for molecula in lst_files:
    molecule = Molecule.from_file(diretorio_treino + arq + "/" + molecula)
    df_aux_tmp = df_aux[df_aux['molecula_config'] == str.replace(molecula, ".xyz", "")]

    val_target = df_aux_tmp[prop].values[0] #target
    val_dist = Distance_Matrix(molecule.coordinates, 23)
    val_features = FeatureMatrix(molecule.symbols, 23)
    val_mol_size = len(molecule.symbols)

    if conjunto == 'treino':
      lst_target_treino.append(val_target)
      lst_features_treino.append(val_features)
      lst_distancias_treino.append(val_dist)
      lst_mol_sizes_treino.append(val_mol_size)

      lst_moleculas_treino.append(arq)
      aux_iso, aux_conf = ExtractIsomerosConfomeros(molecula)
      lst_isomero_treino.append(aux_iso)
      lst_conformero_treino.append(aux_conf)
    else:
      lst_target_valid.append(val_target)
      lst_features_valid.append(val_features)
      lst_distancias_valid.append(val_dist)
      lst_mol_sizes_valid.append(val_mol_size)

      lst_moleculas_valid.append(arq)
      aux_iso, aux_conf = ExtractIsomerosConfomeros(molecula)
      lst_isomero_valid.append(aux_iso)
      lst_conformero_valid.append(aux_conf)

fim = time.time()
print("Time to load data :", fim - inicio)

100%|██████████| 6591/6591 [28:44<00:00,  3.82it/s]

Time to load data : 1724.1846952438354


In [ ]:
df_treino_mapping = pd.DataFrame()
df_treino_mapping['molecula'] = lst_moleculas_treino
df_treino_mapping['isomero'] = lst_isomero_treino
df_treino_mapping['conformero'] = lst_conformero_treino
df_treino_mapping_nodup = df_treino_mapping.drop_duplicates()

df_valid_mapping = pd.DataFrame()
df_valid_mapping['molecula'] = lst_moleculas_valid
df_valid_mapping['isomero'] = lst_isomero_valid
df_valid_mapping['conformero'] = lst_conformero_valid
df_valid_mapping_nodup = df_valid_mapping.drop_duplicates()

In [ ]:
print("Number of molecules structures for training:", len(lst_target_treino))

Number of molecules structures for training: 1069260


# Model Training

In [ ]:
obj_ranker = RankerModel.ModelRanker(loss_f)

In [ ]:
ranker_model = obj_ranker.AtomNeuralNet(loss_f, predict_aggregation = 'sum')

In [ ]:
obj_ranker.TrainModel(ranker_model,
           lst_features_treino, lst_target_treino,
           lst_distancias_treino, lst_mol_sizes_treino,
           df_treino_mapping_nodup, df_treino_mapping,
           lst_features_valid, lst_target_valid,
           lst_distancias_valid, lst_mol_sizes_valid,
           df_valid_mapping_nodup, df_valid_mapping,
           loss_f, 30)

Epoch: 1 Train Loss: 0.03730289       Validation Loss: 0.07884085       Time(s): 1827.7797968387604 
Epoch: 2 Train Loss: 0.16264382       Validation Loss: 0.04080928       Time(s): 1819.8108203411102   Weights Updated :-)
Epoch: 3 Train Loss: 0.07252039       Validation Loss: 0.06700106       Time(s): 1795.9541568756104 
Epoch: 4 Train Loss: 0.10386878       Validation Loss: 0.019489884       Time(s): 1813.3761484622955   Weights Updated :-)
Epoch: 5 Train Loss: 0.023734625       Validation Loss: 0.03460359       Time(s): 1817.2254700660706 
Epoch: 6 Train Loss: 0.036476932       Validation Loss: 0.049277455       Time(s): 1826.138786315918 
Epoch: 7 Train Loss: 0.09681541       Validation Loss: 0.011133475       Time(s): 1818.8593747615814   Weights Updated :-)
Epoch: 8 Train Loss: 0.034830593       Validation Loss: 0.15625577       Time(s): 1781.4324975013733 
Epoch: 9 Train Loss: 0.008197429       Validation Loss: 0.11354179       Time(s): 1771.9139280319214 
Epoch: 10 Train Loss: 

In [ ]:
!mkdir "/content/gdrive/MyDrive/ModelParameters_Pairwise/{prop}_{loss_f}"

In [ ]:
!cp -r "./parameters" "/content/gdrive/MyDrive/ModelParameters_Pairwise/{prop}_{loss_f}/"

In [ ]:
from google.colab import runtime
runtime.unassign()